In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from textblob import TextBlob

In [4]:
data = pd.read_csv('/content/HumanAnnotatedDataset.csv')
print(data)
data.Reviews=data.Reviews.astype(str)

#Transform text to lowercase
data['Reviews'].apply(lambda x: x.lower()) 
#Removing all punctuations and special characters
data['Reviews'] = data['Reviews'].apply(lambda x: re.sub('[^a-zA-z0-9\s]', '', x)) 

     Column1                Company  \
0          0                Netflix   
1          1                   Uber   
2          2                   Yext   
3          3                   CACI   
4          4          Covenant Eyes   
..       ...                    ...   
117      117  Reynolds and Reynolds   
118      118                  Sabre   
119      119  Legrand North America   
120      120    Samsung Electronics   
121      121         Teladoc Health   

                                               Reviews HUMAN_LABEL  
0    high performance culture challenge netflix ask...    positive  
1    pay gone years first started pay great! gas pr...    negative  
2    excellent ceo -lucky worked given opportunity ...    negative  
3    nice place work great company work benefits gr...    negative  
4    best company i’ve ever worked covenant eyes ca...    positive  
..                                                 ...         ...  
117  5 stars best part working company? manageme

StopWord Removal

In [5]:
english_stopwords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
stopwords_list = english_stopwords
data['Reviews'] = data['Reviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_list)]))

In [28]:
data.head()

,Column1,Company,Reviews,HUMAN_LABEL
0,0,Netflix,high performance culture challenge netflix ask...,positive
1,1,Uber,pay gone years first started pay great gas pri...,negative
2,2,Yext,excellent ceo lucky worked given opportunity f...,negative
3,3,CACI,nice place work great company work benefits gr...,negative
4,4,Covenant Eyes,best company ive ever worked covenant eyes car...,positive


In [8]:
# count = 0
# for index in data.index:
#   if data['HUMAN_LABEL'][index] == "positive":
#     count += 1
# print(count)


46


In [19]:
X = data.iloc[:, 2]
y = data.iloc[:, -1]

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

In [21]:
X_train

15     low paced environment day consisted learning c...
72     good place work fun caring people guess starte...
22     good wlb terrible culture never work later sch...
116    overall great place intern first internship co...
91     fun workplace ive worked almost two years hone...
                             ...                        
106    good people challenges sort great product visi...
14     excellent employer microsoft excellent employe...
92     great place work great worklife balance welcom...
51     flexible flexible hours management help sucked...
102    disconnected benefits mediocre high turnaround...
Name: Reviews, Length: 91, dtype: object

In [22]:
X_test

18     productive fun place work would definitely rec...
45     good work ethics working help client stay clea...
47     productive strong work environment good workli...
89     much game culture dedicated product ip making ...
4      best company ive ever worked covenant eyes car...
40     good place work people passionate products tak...
62     great place work great place work never job tr...
107    small company feel good people small company f...
31     overall great place work overall great place w...
55     okay often left manage difficult accounts cand...
53     feel well cared working gdms nature work fulfi...
119    best place work managers jobs correctly always...
10     productive fun environment overall kasisto fun...
90     electrical repair technician great place work ...
109    great wonderful work environment everyone real...
11     productive fun workplace learned new things co...
76     good pay terrible management management terrib...
56     fast moving environment 

TfidVectorization

In [23]:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer

my_stop_words = text.ENGLISH_STOP_WORDS
# Create feature vectors
vectorizer = TfidfVectorizer(min_df=5,
                             max_df=0.95,
                             stop_words=my_stop_words,
                             sublinear_tf = True,
                             use_idf = True,
                             ngram_range=(1,1))
train_vectors = vectorizer.fit_transform(X_train.values.astype('U'))
test_vectors = vectorizer.transform(X_test.values.astype('U'))

Random Forest

In [24]:
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
# Perform classification with SVM, kernel=linear
classifier = RandomForestClassifier()
t0 = time.time()
classifier.fit(train_vectors, y_train)
t1 = time.time()
prediction = classifier.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(y_test, prediction, output_dict=True,labels=np.unique(prediction))
print('positive: ', report['positive'])
print('negative: ', report['negative'])


Training time: 0.190714s; Prediction time: 0.015461s
positive:  {'precision': 0.8181818181818182, 'recall': 0.6923076923076923, 'f1-score': 0.7500000000000001, 'support': 13}
negative:  {'precision': 0.8, 'recall': 0.8888888888888888, 'f1-score': 0.8421052631578948, 'support': 18}


K-Nearest Neighbors

In [26]:
from sklearn.neighbors import KNeighborsClassifier
classifier = KNeighborsClassifier()
t0 = time.time()
classifier.fit(train_vectors, y_train)
t1 = time.time()
prediction = classifier.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1
# results
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(y_test, prediction, output_dict=True)
print('positive: ', report['positive'])
print('negative: ', report['negative'])


Training time: 0.002733s; Prediction time: 0.008648s
positive:  {'precision': 0.7, 'recall': 0.5384615384615384, 'f1-score': 0.608695652173913, 'support': 13}
negative:  {'precision': 0.7142857142857143, 'recall': 0.8333333333333334, 'f1-score': 0.7692307692307692, 'support': 18}
